### Programatically Start Core Machine with a Startup Script and Stop Core Machines using Paperspace API

- This notebook demonstrates how to start and stop a Paperspace Core machine using the Paperspace API.

- Make sure you have your API token created using this instruction: https://docs.digitalocean.com/reference/paperspace/api-keys/

- Also find the Paperspace API reference here: https://docs.digitalocean.com/reference/paperspace/pspace/api-reference/

### Example Use Case:
- We have setup a Paperspace Core machine running a Mistral-7B vLLM Docker container and exposes the API on Port 8000

- Say, there is no constant usage of the machine and we want to stop the machine when not in use and start it when needed.

- We can do this by using the Paperspace API as shown in the diagram below 

<!-- ![Core_PS_API Image](core_api_flow_start_stop.png) -->
<img src="core_api_flow_start_stop.png" width="600" height="450">

In [56]:
import requests
import json
import time

api_key = "PAPERSPACE_API_KEY"
hf_token = "HF_READ_TOKEN"
machine_id = "MACHINE_ID"

In [23]:

def get_machine_status(machine_id, auth_key):
    url = "https://api.paperspace.com/v1/machines/"
    headers = {
    'Authorization': 'Bearer '+ auth_key
    }
    url += machine_id
    response = requests.request("GET", url, headers=headers)
    response_obj = json.loads(response.text)
    # print(response_obj.keys())
    # print(response_obj['state']) 
    return response_obj['state'] # should be "off" "starting" "stopping" "restarting" "serviceready" "ready" "upgrading" "provisioning"


In [75]:
get_machine_status(machine_id, api_key)

'off'

In [41]:
def stop_machine(machine_id, auth_key):
    url = "https://api.paperspace.com/v1/machines/"
    headers = {
    'Authorization': 'Bearer '+ auth_key
    }
    url += machine_id + "/stop"
    response = requests.request("PATCH", url, headers=headers)
    if response.status_code == 200:
        return True
    response_obj = json.loads(response.text)
    print(response_obj)
    return False

In [74]:
stop_machine(machine_id, api_key)

True

In [43]:
def start_machine(machine_id, auth_key):
    url = "https://api.paperspace.com/v1/machines/"
    headers = {
    'Authorization': 'Bearer '+ auth_key
    }
    url += machine_id + "/start"
    response = requests.request("PATCH", url, headers=headers)
    if response.status_code == 200:
        return True
    response_obj = json.loads(response.text)
    print(response_obj)
    return False

In [44]:
start_machine(machine_id, api_key)

True

In [72]:
def create_new_startup_script_from_shell_file(name, shell_fp, run_on_every_boot, api_key):
    with open(shell_fp, 'r') as f:
        shell_script = f.read()
    shell_script = shell_script.replace("hf_readtoken", hf_token)
    # print(shell_script)
    startup_script = {
        "name": name,
        "script": shell_script,
        "isRunOnce": not run_on_every_boot
    }
    url = "https://api.paperspace.com/v1/startup-scripts"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer '+ api_key,
    }
    payload = json.dumps(startup_script)
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response)
    if response.status_code == 200:
        response_obj = json.loads(response.text)
        return response_obj['id']
    else:
        response_obj = json.loads(response.text)
        print(response_obj)
        return None

In [76]:
ss_id = create_new_startup_script_from_shell_file("start_vllm_docker_hf_ss", "example_script.sh", True, api_key)
print("Startup Script ID:", ss_id)

<Response [200]>
Startup Script ID: scor8zqtf


In [54]:
def assign_startup_script_to_machine(machine_id, startup_script_id, api_key):
    url = f"https://api.paperspace.com/v1/startup-scripts/{startup_script_id}/assign"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer '+ api_key,
    }
    payload = json.dumps({"machineId": machine_id})
    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code == 200:
        return True
    response_obj = json.loads(response.text)
    print(response_obj)
    return False

In [77]:
assign_startup_script_to_machine(machine_id, ss_id, api_key)

True

In [79]:
def start_and_wait_for_machine_to_be_ready(machine_id, api_key, max_wait_time=300):
    if start_machine(machine_id, api_key):
        print("Machine starting...")
        start_time = time.time()
        while get_machine_status(machine_id, api_key) != "ready":
            time.sleep(10)
            current_time = time.time()
            if current_time - start_time > max_wait_time:
                print("Machine failed to start even after waiting for max time:", max_wait_time)
                break
        print("Machine is ready!")
    else:
        print("Machine failed to start")


In [78]:
start_and_wait_for_machine_to_be_ready(machine_id, api_key)

Machine starting...
Machine ready


#### NOTE: Once the Machine is ready, the startup script might still take some time to run, so wait for a few minutes before checking if the service is ready
